In [ ]:
'''
In this file 
We do data processing for SketchVAE training
Since it requires rhythm and pitch tokens, we should process the previous data into the rhythm and pitch data first.
'''
from loader.dataloader import DataLoader
import numpy as np

data_path = [
    "data/irish_train.npy", 
    "data/irish_validate.npy", 
    "data/irish_test.npy"
]

In [ ]:
# import data
train_x = np.load(data_path[0],allow_pickle = True)
validate_x = np.load(data_path[1],allow_pickle = True)
test_x = np.load(data_path[2],allow_pickle = True)


In [ ]:
# note extraction
hold_state = 128
rest_state = 129
def extract_note(x, pad_token = 128):
    d = []
    for i in x:
        if i < 128:
            d.append(i)
    ori_d = len(d)
    d.extend([pad_token] * (len(x) - len(d)))
    return np.array(d), ori_d

def extract_rhythm(x, hold_token = 2, rest_token = 3):
    d = []
    for i in x:
        if i < 128:
             d.append(1)
        elif i == hold_state:
             d.append(hold_token)
        else:
             d.append(rest_token)
    return np.array(d)
             

In [ ]:
# process rhythm and pitch tokens
split_size = 24
new_data = []
# change here to be train_x/validate_x/test_x
for i,d in enumerate(train_x):
    d = np.array(d["notes"])
    ds = np.split(d, list(range(split_size,len(d),split_size)))
    data = []
    for sd in ds:
        if len(sd) != split_size:
            continue
        q,k = extract_note(sd)
        if k == 0:
            continue
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
    if i % 1000 == 0:
        print("processed:", i)

In [ ]:
# now you can see what processed data looks like [gd, pitch, rhythm, pitch_len]
print(new_data[0][0])

In [ ]:
# we need to extract each measure in each song
final_data = []
for d in new_data:
    for dd in d:
        final_data.append(dd)
print(len(final_data))

In [ ]:
# save the data named: irish_xxx_chord_rhythm.npy
np.save("data/irish_train_chord_rhythm.npy",final_data)